In [ ]:
# 网络摄像头代码，用于采集数据（图像）
# 代码编写：谢作如(2021.4.12)

import remi.gui as gui
from remi import start, App
import time,threading
import qrcode,base64,io,cv2
import gradio as gr
import BaseDeploy as bd
import matplotlib.pyplot as plt

model_path = 'E:\\Lindia\\2023.08.21 AI Lab 夏令营\\视力检测\\out_file\\hand.onnx'
model = bd(model_path)

camera = cv2.VideoCapture(0)
# 临时变量，用于保存画面
tempcam = None
# 图片名称编号
f_num = 1

#全局变量控制进程
running=0


# 返回摄像头画面，自定义大小
def get_frames(x,y):
    global tempcam
    success, frame = camera.read()  # read the camera frame
    if success:
        tempcam = frame #临时变量获取摄像头画面
        frame = cv2.flip(frame, 1, dst=None) #镜像
        frame = cv2.resize(frame, (x, y), interpolation=cv2.INTER_LINEAR) #自定义大小
        image = cv2.imencode('.jpg', frame)[1] #解码成图像格式
        base64_data = str(base64.b64encode(image))[2:-1]
        data='data:image/jpeg;base64,%s'%base64_data
        return data
    else:
        return get_img("摄像头启动失败")

# Web主程序
class MyApp(App):
    def __init__(self, *args):
        super(MyApp, self).__init__(*args)

    def main(self):
        #页面大小
        container = gui.VBox(width=700, height=450,style={'margin':'0px auto'}) 
        
        
        #页面元素
        bts = gui.HBox(width=300, height=50)
        self.bt_start = gui.Button('[ Start ]')
        self.bt_end = gui.Button('[ End ]')
        bts.append(self.bt_start)
        bts.append(self.bt_end)
        
        pic = gui.HBox(width=350, height=100)
        self.stream = gui.Image(get_frames(150,100)) #实施显示摄像头画面
        self.photo = gui.Image(get_frames(150,100)) #显示拍摄照片        
        pic.append(self.stream)
        pic.append(self.photo)
        
        self.lbl_01 = gui.Label('Web摄像头画面采集器',style={'font-size': '25px'}) #标题
        self.lbl_running = gui.Label('Stopped')
        self.result = gui.Label('Result will show here..') #预测结果

        
        # 添加到网页上
        container.append(self.lbl_01)
        container.append(pic)
        container.append(self.lbl_running)
        container.append(self.result)        
        container.append(bts)
        
        # 按钮按下时执行
        self.bt_start.onclick.do(self.on_button_pressed,'start')
        self.bt_end.onclick.do(self.on_button_pressed,'end')
        
        # 开启新的进程刷新画面
        stream = threading.Thread(target=self.showimg)
        stream.start()
        
        # 开启新的进程自动拍照
        capture = threading.Thread(target=self.capture)
        capture.start()
        

        # returning the root widget
        return container
    
    #-------------------------------------------------------------------------
    def on_button_pressed(self, emitter, tag):
        global running
        if tag=='start':
            running=1
            self.lbl_running.set_text("Running..")
            print(running)
        if tag=='end':
            running=0
            self.lbl_running.set_text("Stopped")
            print(running)
    
    def capture(self):
        while True:
            if not running:
                continue
            test = tempcam
            test = cv2.flip(test, 1, dst=None)
            frame = test
            frame = cv2.resize(frame, (150,100), interpolation=cv2.INTER_LINEAR) #自定义大小
            image = cv2.imencode('.jpg', frame)[1] #解码成图像格式
            base64_data = str(base64.b64encode(image))[2:-1]
            data='data:image/jpeg;base64,%s'%base64_data
            self.photo.set_image(data)
            result = model.inference(test)
            result = model.print_result(result)
            print(result)
            self.result.set_text(result['预测结果'])
            time.sleep(3)
    
    # 刷新网页的摄像头画面
    def showimg(self):
        while True:
            if not running:
                continue                            
            self.stream.set_image(get_frames(150,100))
            time.sleep(0.2)

# starts the web server
start(MyApp,title='摄像头实时视频',address='0.0.0.0')


Success load model info generate by MMEdu>=0.1.15: {"codebase": "MMCls", "modelname": "LeNet", "classes": ["down", "left", "right", "up"]}


remi.server      INFO     Started httpserver http://0.0.0.0:56384/
remi.request     INFO     built UI (path=/)
127.0.0.1 - - [25/Aug/2023 11:51:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2023 11:51:42] "GET /res:style.css HTTP/1.1" 200 -
remi.server.ws   INFO     connection established: ('127.0.0.1', 56393)
127.0.0.1 - - [25/Aug/2023 11:51:42] "GET /res:font.woff2 HTTP/1.1" 200 -
remi.server.ws   INFO     handshake complete


1
{'标签': 0, '置信度': 0.68699145, '预测结果': 'down'}
{'标签': 0, '置信度': 0.61691993, '预测结果': 'down'}
{'标签': 0, '置信度': 0.6590748, '预测结果': 'down'}
{'标签': 0, '置信度': 0.68505526, '预测结果': 'down'}
{'标签': 0, '置信度': 0.69077885, '预测结果': 'down'}
{'标签': 0, '置信度': 0.69580054, '预测结果': 'down'}
{'标签': 0, '置信度': 0.5470443, '预测结果': 'down'}
{'标签': 0, '置信度': 0.590666, '预测结果': 'down'}
{'标签': 0, '置信度': 0.64789194, '预测结果': 'down'}
0


remi.server.ws   ERROR    Error managing incoming websocket message
Traceback (most recent call last):
  File "C:\Users\Lindia Chen\AppData\Roaming\Python\Python39\site-packages\remi\server.py", line 167, in read_next_message
    head = self.rfile.read(2)
  File "D:\Anaconda\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
